# Get the table from wikipedia link

In [1]:
import requests
import lxml.html as lh
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))


# Clean the table data

In [37]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

[len(C) for (title,C) in col]
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
#df2=df.dropna(subset=['Borough'], axis=0) didn't work
df2 = df[df.Borough != 'Not assigned']
df2.index = range(len(df2))
df2.iloc[:,2] = df2.iloc[:,2].str[:-1] #remove last character of a string
df2.columns = df2.columns.str.replace('\n','')
df2.head(10)



,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


# Now set missing Neighbourhood value to Borought name

In [19]:

for i in range(0,df2.shape[0]):
    if df2.iloc[i, 2] == 'Not assigned':
        df2.iloc[i, 2] = df2.iloc[i, 1]
#iloc is used for index system
df2.head(10)
df2.to_csv('test1.csv', index=False)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


# Group the table by Postcode and join Neighbourhood text

In [47]:
df3=df2.groupby(['Postcode','Borough'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else ' , '.join(x))
df3

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge , Malvern , Rouge , Malvern , Rouge , Ma..."
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union , Hig..."
2,M1E,Scarborough,"Guildwood , Morningside , West Hill , Guildwoo..."
3,M1G,Scarborough,"Woburn , Woburn , Woburn"
4,M1H,Scarborough,"Cedarbrae , Cedarbrae , Cedarbrae"
5,M1J,Scarborough,"Scarborough Village , Scarborough Village , Sc..."
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park ..."
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge , Clairlea ,..."
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village W..."
9,M1N,Scarborough,"Birch Cliff , Cliffside West , Birch Cliff , C..."


# Export Results in csv

In [43]:
df3.to_csv('export_dataframe1.csv', index=False)